# XML Parser -- Richards

In [2]:
# Import necessary libraries.
import re, glob, csv, sys, os
import pandas as pd
import xml.etree.ElementTree as ET

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/"

# Gather all .xml files using glob.
list_of_files = glob.glob(abs_dir + "Data/PSC/Richards/ESR-XML-Files-MHS/*.xml")

## Define Functions

In [3]:
'''
Arguments of Functions:

    namespace:

    ancestor:
    
    xpath_as_string:
    
    attrib_val_str:
    
'''

# Read in file and get root of XML tree.
def get_root(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root


# Get namespace of individual file from root element.
def get_namespace(root):
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    return ns

## Parse Documents

In [11]:
%%time

dataframe = []

for file in list_of_files:
    
    try:
        root = get_root(file)
        ns = get_namespace(root)

        reFile = str(re.search(r'.*/(.*.xml)', str(file)).group(1)) # get filename without path

        date = root.find('.//ns:date/[@type="creation"]', ns).get('when') # get date.

        source = root.find('.//ns:bibl//ns:author', ns).text   # get source/author & target/recipient
        target = root.find('.//ns:bibl//ns:recipient', ns).text
        
    #     Loops
    #     loop to get all references (persRef)
        references_l = []
        for ref in root.findall('.//ns:persRef', ns):
            person = ref.get('ref')
            references_l.append(person)
        references = ','.join(references_l)

    #     loop to get subjects.
        subject_l = []
        for subject in root.findall('.//ns:subject', ns):
            subject_l.append(subject.text)
        subjects = ','.join(subject_l)

    #     loop to get all text within <div type="docbody">
        text_l = []
        for txt in root.findall('.//ns:div[@type="docbody"]', ns):
            string = ''.join(ET.tostring(txt, encoding='unicode', method='text'))
            clean_string = re.sub(r'[\t\n\s]+', ' ', string)
            text_l.append(clean_string)
        content = ' '.join(text_l)


        row = {'file': reFile, 'date': date, 'source': source, 'target':target, 
               'subjects': subjects, 'references': references, 'text': content}
        

        dataframe.append(row)
        
    except:
        print (file, '\n')
    
df = pd.DataFrame(dataframe)

df.head(3)

/Users/quinn.wi/Documents/Data/PSC/Richards/ESR-XML-Files-MHS/ESR-EDA-1893-09-24.xml 

CPU times: user 7.85 ms, sys: 2.02 ms, total: 9.87 ms
Wall time: 8.87 ms


,file,date,source,target,subjects,references,text
0,ESR-EDA-1892-01-08.xml,1892-01-08,richards-ellen,atkinson-edward,"1893 Chicago World's Fair,Aladdin Oven,New Eng...","palmer-bertha,hovey-e,daniells-unknown",Boston Jan 8 1892 My dear Mr Atkinson I enclo...
1,ESR-EDA-1892-04-12.xml,1892-04-12,richards-ellen,atkinson-edward,"Aladdin Oven,nutrition,cooking",abel-mary,April 12— Dear Mr Atkinson I expect Mrs Abel ...
2,ESR-EDA-1892-04-07.xml,1892-04-07,richards-ellen,atkinson-edward,"Aladdin Oven,Nutrition,cooking","conro-emma,abel-mary","Boston, April 7, 1892 My dear Mr. Atkinson I ..."


In [12]:
%%time

df.to_csv(abs_dir + 'Data/Output/ParsedXML/Richards_dataframe.txt', sep='\t', index=False)

CPU times: user 2.42 ms, sys: 1.7 ms, total: 4.11 ms
Wall time: 3.46 ms
